In [196]:
import pandas as pd
from collections import Counter, defaultdict
import tensorflow as tf
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [201]:
folder_name = "/home/sanala/Juputer try/PET/data/"
save_name="/home/sanala/Juputer try/PET/data-fsl/"
for i in range(10):
    language = ['amh','eng','fra','hau','ibo','lin','pcm','run','swa','yor'][i]
 
    train = pd.read_csv(f'{folder_name}/{language}/train.tsv',sep='\t')
    dev= pd.read_csv(f'{folder_name}/{language}/dev.tsv',sep='\t')
    test= pd.read_csv(f'{folder_name}/{language}/dev.tsv',sep='\t')
    train_nodup=train.drop_duplicates(subset=['headline'], keep='first')
    dev_nodup=dev.drop_duplicates(subset=['headline'], keep='first')
    classes = train['category'].unique()
    samples_per_class=50 #5 10 20 50
    # create an empty list to store the randomly selected samples
    random_samples = []

    # iterate through each class
    for class_name in classes:
        # select all rows with the current class
        class_rows = train_nodup[train_nodup['category'] == class_name]
        # randomly select X rows from the class_rows DataFrame
        random_class_samples = class_rows.sample(samples_per_class)
        # add the randomly selected samples to the list
        random_samples.append(random_class_samples)
    random_samples_df = pd.concat(random_samples).sample(frac=1)
    new_train=random_samples_df.reset_index(drop=True)
    unlabeled_train = train.drop(new_train.index)
    
    random_samples = []
    samples_per_class=5
    for class_name in classes:
        class_rows = dev_nodup[dev_nodup['category'] == class_name]
        random_class_samples = class_rows.sample(samples_per_class)
        random_samples.append(random_class_samples)
    random_samples_df = pd.concat(random_samples).sample(frac=1)
    new_dev=random_samples_df.reset_index(drop=True)
    
    unlabeled_train.insert(0, 'unlabeled', 'unlabeled')
    unlabeled_train.reset_index(drop=True)
    unlabeled_train.drop(columns='category', inplace=True)
    
    new_train.to_csv(f'{save_name}/{language}/50sample/train.tsv', sep="\t", index=False  ,header=False)
    unlabeled_train.to_csv(f'{save_name}/{language}/50sample/unlabeled.tsv', sep="\t", index=False,header=False)
    new_dev.to_csv(f'{save_name}/{language}/50sample/dev.tsv', sep="\t", index=False,header=False)

#new_train



In [202]:
 
for i in range(10):
    language = ['amh','eng','fra','hau','ibo','lin','pcm','run','swa','yor'][i]
 
    train = pd.read_csv(f'{folder_name}/{language}/train.tsv',sep='\t')
    dev= pd.read_csv(f'{folder_name}/{language}/dev.tsv',sep='\t')
    test= pd.read_csv(f'{folder_name}/{language}/dev.tsv',sep='\t')
    print(language)
    print(train['category'].value_counts())

#new_train


amh
health      350
politics    350
sports      329
business    282
Name: category, dtype: int64
eng
sports           700
politics         562
business         546
health           519
entertainment    514
technology       412
Name: category, dtype: int64
fra
health        350
sports        350
politics      350
business      350
technology     76
Name: category, dtype: int64
hau
entertainment    350
politics         350
sports           347
religion         345
health           345
business         279
technology       203
Name: category, dtype: int64
ibo
politics         350
health           296
entertainment    256
business         204
sports           199
religion          51
Name: category, dtype: int64
lin
politics    350
health      135
sports       66
business     57
Name: category, dtype: int64
pcm
sports           344
entertainment    322
politics         216
health           111
business          67
Name: category, dtype: int64
run
politics         350
sports           293
h

In [170]:

train = pd.read_csv('/home/sanala/Juputer try/PET/data/fra/train.tsv', sep="\t")
dev = pd.read_csv('/home/sanala/Juputer try/PET/data/fra/dev.tsv', sep="\t")
test = pd.read_csv('/home/sanala/Juputer try/PET/data/fra/test.tsv', sep="\t")

train.info()
train.head(10)
dev.info()     

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1476 entries, 0 to 1475
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  1476 non-null   object
 1   headline  1476 non-null   object
 2   text      1476 non-null   object
 3   url       1476 non-null   object
dtypes: object(4)
memory usage: 46.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  211 non-null    object
 1   headline  211 non-null    object
 2   text      211 non-null    object
 3   url       211 non-null    object
dtypes: object(4)
memory usage: 6.7+ KB


In [171]:
#There are repeated examples in the training dataset
print('Unique comments: ', train.iloc[:,1].nunique()==train.shape[0] )
print('Null values: ', train.isnull().values.any())
     

Unique comments:  False
Null values:  False


In [172]:
train['category'].value_counts()

health        350
sports        350
politics      350
business      350
technology     76
Name: category, dtype: int64

In [173]:
#Usually the headlines are sort but outliers can appear.
print('average sentence length: ', train.iloc[:,1].str.split().str.len().mean())
print('stdev sentence length: ', train.iloc[:,1].str.split().str.len().std())
print('max sentence length: ', train.iloc[:,1].str.split().str.len().max())

average sentence length:  12.409214092140921
stdev sentence length:  3.760529595623102
max sentence length:  33


In [174]:
train[train.iloc[:,1].duplicated(keep=False).values]

,category,headline,text,url
914,business,De vendeur de pain à roi de la pizza,Dans Questions d'Argent cette semaine : Retrou...,https://www.bbc.com/afrique/region-53501527
933,business,"Le souchet, tubercule sauvage devenu ""super al...",Le souchet est un amuse-bouche qu’on peut trou...,https://www.bbc.com/afrique/articles/cv2rg2xe4qyo
1096,health,"Le souchet, tubercule sauvage devenu ""super al...",Le souchet est un amuse-bouche qu’on peut trou...,https://www.bbc.com/afrique/articles/cv2rg2xe4qyo
1427,business,De vendeur de pain à roi de la pizza,Dans Questions d'Argent cette semaine : Retrou...,https://www.bbc.com/afrique/region-53501527


In [175]:
train_nodup=train.drop_duplicates(subset=['headline'], keep='first')


In [176]:
train_nodup

,category,headline,text,url
0,sports,"L'Ouganda à l'assaut des ""fimbu"" de la RDC","L'Ouganda, placé 79e au classement FIFA le 4 a...",https://www.bbc.com/afrique/sports-48730747
1,business,Stopper la détérioration de l’environnement po...,La responsable de la biodiversité des Nations ...,https://www.bbc.com/afrique/articles/cevdz4xrp42o
2,sports,Coupe d'Afrique des nations 2022 : le sélectio...,"Le sélectionneur de la Sierra Leone, John Keis...",https://www.bbc.com/afrique/region-59881445
3,business,Tissus africains : pourquoi les teintureries h...,"Depuis plus de six siècles, une vaste zone sit...",https://www.bbc.com/afrique/region-61202726
4,business,Les revenus pendant la pandémie des dix hommes...,"Pendant la pandémie de coronavirus, la richess...",https://www.bbc.com/afrique/monde-55812018
...,...,...,...,...
1471,sports,C1: Le PSG va disputer sa toute première finale,Le Paris SG s'est qualifié pour sa toute premi...,https://www.bbc.com/afrique/sports-53829969
1472,sports,"Aubameyang invité à rejoindre un ""club plus am...","L'attaquant gabonais d'Arsenal, dont le contra...",https://www.bbc.com/afrique/sports-52262194
1473,politics,Guerre Ukraine - Russie : qui est Sergey Surov...,La désignation de Sergey Surovikin pour dirige...,https://www.bbc.com/afrique/monde-63228038
1474,technology,"Svetlana Jitomirskaya, la mathématicienne à l'...",Il existe un type de papillon qui captive le p...,https://www.bbc.com/afrique/monde-63956646


In [177]:
dev[dev.iloc[:,1].duplicated(keep=False).values]

,category,headline,text,url


In [178]:
dev_nodup=dev.drop_duplicates(subset=['headline'], keep='first')
 

In [163]:
dev_nodup

,category,headline,text,url
0,sports,ዘጠኝ ግብ በተስተናገደበት ጨዋታ ማንቸስተር ሲቲ ድል ተቀዳጅቷል,በፈረንጆቹ የገና በዓል ማግስት ማንቸስተር ሲቲዎች ለዘጠነኛ ተከታታይ ጊዜ...,https://www.bbc.com/amharic/news-59799565
1,sports,ለአውሮፓ ሻምፒዮንነት ቀሪዎቹ ስምንት አገራት ፍልሚያቸውን ዛሬ ይጀምራሉ,ለአውሮፓ ዋንጫ አሸናፊነት የሚፎካከሩት ቀሪዎቹ ስምንት አገራት ፍልሚያቸው...,https://www.bbc.com/amharic/news-57693522
2,business,ከባድ ፍንዳታ ያጋጠመው የክሬሚያ ድልድይ በከፊል ተከፍቷል - ሩሲያ,በከባድ ፍንዳታ የወደመው እና ከሩሲያ ወደ ክሬሚያ የሚወስደው ብቸኛ ድልድ...,https://www.bbc.com/amharic/articles/cd1gdx1dvjeo
3,business,የአጎዋ የንግድ ዕድል መስተጓጎል በፋብሪካዎችና በሠራተኞች ላይ የፈጠረው ስጋት,የአፍሪካ የእድገትና ዕድል ድንጋጌ በምኅጻረ ቃሉ አጎዋ ተብሎ በሚታወቀው ...,https://www.bbc.com/amharic/news-59297844
4,health,የኮቪድ ህሙማንን የሚያክመው ኤካ ኮተቤ ሆስፒታል እየሞላ ነው ተባለ,የኮቪድ-19 ሕሙማንን ተቀብሎ ህክምና የሚሰጠው ኤካ ኮተቤ ሆስፒታል በኮቪ...,https://www.bbc.com/amharic/news-58671327
...,...,...,...,...
183,politics,ከእስር የተለቀቀው የጋዳፊ ልጅ ቱርክ ገባ,ሦስተኛው የጋዳፊ ልጅ ከሊቢያ እስር ቤት ከተለቀቀ በኋላ ወደ ቱርክ፣ ኢስ...,https://www.bbc.com/amharic/58459657
184,business,የምግብ ዋጋ መናርን ለመቋቋም አማራጮችን እየተገበሩ ያሉ የሦስት አገር ነ...,በዓለም ዙሪያ ተከሰተውን የኮሮናቫይረስ ወረርሽኝን ተከትሎ በተከሰተ ምጣኔ...,https://www.bbc.com/amharic/articles/cjm27vvj2yxo
185,politics,አዲሱ የደኅንነት መሥሪያ ቤቱ “የሥነ ልቦና ጦርነትን የመመከት ሥልጣን’’...,የሥነ ልቦና ጦርነት ቃሉ በራሱ እጅግ አከራካሪ አንዲሁም አተረጓጎሙም የተ...,https://www.bbc.com/amharic/articles/cnl15g4xjg4o
186,sports,ዓለም በጉጉት እየጠበቀው ያለው ወሳኙ የእንግሊዝ እና ፈረንሳይ ጨዋታ,በኳታሩ የዓለም ዋንጫ ውድድር በጉጉት ከሚጠበቁ ጨዋታዎች መካከል ቀዳሚ ሊ...,https://www.bbc.com/amharic/articles/c3g4lrz5ggko


In [179]:
#We take X examples from each label, and the rest become the unlabeled data-set.
classes = train['category'].unique()
samples_per_class=20 #5 10 20 50
# create an empty list to store the randomly selected samples
random_samples = []

# iterate through each class
for class_name in classes:
    # select all rows with the current class
    class_rows = train_nodup[train_nodup['category'] == class_name]
    # randomly select X rows from the class_rows DataFrame
    random_class_samples = class_rows.sample(samples_per_class)
    # add the randomly selected samples to the list
    random_samples.append(random_class_samples)
random_samples_df = pd.concat(random_samples).sample(frac=1)
new_train=random_samples_df.reset_index(drop=True)
unlabeled_train = train.drop(new_train.index)
#new_train
#unlabeled_train

In [180]:
#new_train.columns = [0, 1, 2,3]
#unlabeled_train.columns = [0, 1, 2,3]

In [181]:
random_samples = []
samples_per_class=5
for class_name in classes:
    class_rows = dev_nodup[dev_nodup['category'] == class_name]
    random_class_samples = class_rows.sample(samples_per_class)
    random_samples.append(random_class_samples)
random_samples_df = pd.concat(random_samples).sample(frac=1)
new_dev=random_samples_df.reset_index(drop=True)
#new_dev.columns = [0, 1, 2,3]

In [182]:
unlabeled_train.insert(0, 'unlabeled', 'unlabeled')
unlabeled_train.reset_index(drop=True)

,unlabeled,category,headline,text,url
0,unlabeled,business,"Calinou’nou, la société qui veut redonner aux ...","« J'étais malade, j'ai pris une semaine, à mon...",https://www.bbc.com/afrique/region-50197742
1,unlabeled,health,Ce que les Japonais peuvent nous apprendre sur...,Le Japon a l'une des populations les plus âgée...,https://www.bbc.com/afrique/region-55619408
2,unlabeled,technology,Y-a-t'il un espion caché dans votre poche?,"Pour de nombreuses personnes, leur smartphone ...",https://www.bbc.com/afrique/monde-48828261
3,unlabeled,health,Ebola: ce que l'on sait de la résurgence de la...,La République démocratique du Congo annonce di...,https://www.bbc.com/afrique/region-55980559
4,unlabeled,politics,Le Danemark suspend son aide à la Tanzanie apr...,"Le Danemark suspend 9,8 millions de dollars d'...",https://www.bbc.com/afrique/region-46221076
...,...,...,...,...,...
1371,unlabeled,sports,C1: Le PSG va disputer sa toute première finale,Le Paris SG s'est qualifié pour sa toute premi...,https://www.bbc.com/afrique/sports-53829969
1372,unlabeled,sports,"Aubameyang invité à rejoindre un ""club plus am...","L'attaquant gabonais d'Arsenal, dont le contra...",https://www.bbc.com/afrique/sports-52262194
1373,unlabeled,politics,Guerre Ukraine - Russie : qui est Sergey Surov...,La désignation de Sergey Surovikin pour dirige...,https://www.bbc.com/afrique/monde-63228038
1374,unlabeled,technology,"Svetlana Jitomirskaya, la mathématicienne à l'...",Il existe un type de papillon qui captive le p...,https://www.bbc.com/afrique/monde-63956646


In [183]:
unlabeled_train.drop(columns='category', inplace=True)
unlabeled_train

,unlabeled,headline,text,url
100,unlabeled,"Calinou’nou, la société qui veut redonner aux ...","« J'étais malade, j'ai pris une semaine, à mon...",https://www.bbc.com/afrique/region-50197742
101,unlabeled,Ce que les Japonais peuvent nous apprendre sur...,Le Japon a l'une des populations les plus âgée...,https://www.bbc.com/afrique/region-55619408
102,unlabeled,Y-a-t'il un espion caché dans votre poche?,"Pour de nombreuses personnes, leur smartphone ...",https://www.bbc.com/afrique/monde-48828261
103,unlabeled,Ebola: ce que l'on sait de la résurgence de la...,La République démocratique du Congo annonce di...,https://www.bbc.com/afrique/region-55980559
104,unlabeled,Le Danemark suspend son aide à la Tanzanie apr...,"Le Danemark suspend 9,8 millions de dollars d'...",https://www.bbc.com/afrique/region-46221076
...,...,...,...,...
1471,unlabeled,C1: Le PSG va disputer sa toute première finale,Le Paris SG s'est qualifié pour sa toute premi...,https://www.bbc.com/afrique/sports-53829969
1472,unlabeled,"Aubameyang invité à rejoindre un ""club plus am...","L'attaquant gabonais d'Arsenal, dont le contra...",https://www.bbc.com/afrique/sports-52262194
1473,unlabeled,Guerre Ukraine - Russie : qui est Sergey Surov...,La désignation de Sergey Surovikin pour dirige...,https://www.bbc.com/afrique/monde-63228038
1474,unlabeled,"Svetlana Jitomirskaya, la mathématicienne à l'...",Il existe un type de papillon qui captive le p...,https://www.bbc.com/afrique/monde-63956646


In [135]:

#We take X examples from each label, and the rest become the unlabeled data-set.
#Instead, we could also make the new training data-set following the proportion of examples per label.
#PVP pattern-verbalizer pair (PVP)

In [184]:
new_train.to_csv("/home/sanala/Juputer try/PET/data-fsl/fra/20sample/train.tsv", sep="\t", index=False  ,header=False)
unlabeled_train.to_csv("/home/sanala/Juputer try/PET/data-fsl/fra/20sample/unlabeled.tsv", sep="\t", index=False,header=False)
new_dev.to_csv("/home/sanala/Juputer try/PET/data-fsl/fra/20sample/dev.tsv", sep="\t", index=False,header=False)

### Train your own PET



In [224]:
%cd /home/sanala/Juputer try/pet/


/home/sanala/Juputer try/pet
